# FastMCP Agent Databricks Notebook

This notebook demonstrates how to connect to a FastMCP server, list available tools, and invoke them using the agent pattern. It is designed for Databricks environments.

In [ ]:
# Databricks: Install dependencies (run once per cluster)
# %pip install langchain-mcp-adapters langgraph langchain langchain-openai openai python-dotenv mlflow

In [ ]:
import os
import asyncio
import logging
from dotenv import load_dotenv
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain.chat_models import init_chat_model

# Load environment variables from Databricks secrets or .env if running locally
load_dotenv()
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s %(message)s')

In [ ]:
# Databricks: Set secrets as environment variables (example)
# dbutils.widgets.text('FASTMCP_URL', '')
# os.environ['FASTMCP_URL'] = dbutils.widgets.get('FASTMCP_URL')
# os.environ['AZURE_OPENAI_MODEL'] = dbutils.secrets.get(scope='my-scope', key='AZURE_OPENAI_MODEL')
# os.environ['AZURE_OPENAI_API_KEY'] = dbutils.secrets.get(scope='my-scope', key='AZURE_OPENAI_API_KEY')
# os.environ['AZURE_OPENAI_ENDPOINT'] = dbutils.secrets.get(scope='my-scope', key='AZURE_OPENAI_ENDPOINT')
# os.environ['AZURE_OPENAI_API_VERSION'] = dbutils.secrets.get(scope='my-scope', key='AZURE_OPENAI_API_VERSION')

In [ ]:
async def list_and_test_tools():
    fastmcp_url = os.getenv('FASTMCP_URL', 'http://localhost:8000/mcp')
    logging.info(f'Connecting to FastMCP server at {fastmcp_url}')
    client = MultiServerMCPClient({
        'sfmc': {
            'url': fastmcp_url,
            'transport': 'streamable_http',
        }
    })
    try:
        tools = await client.get_tools()
        print('Available tools:')
        for tool in tools:
            print(f'- {tool.name}: {getattr(tool, "description", "")}')
        # Try invoking each tool
        for tool in tools:
            args = {}
            if tool.name == 'demo_echo':
                args = {'input': 'test input'}
            print(f'Invoking {tool.name}...')
            try:
                result = await tool.ainvoke(args)
                print(f'Result from {tool.name}: {result}')
            except Exception as e:
                print(f'Error invoking {tool.name}: {e}')
    except Exception as e:
        print(f'Failed to connect or list tools: {e}')

In [ ]:
# Databricks: Run the async agent logic
await list_and_test_tools()